### Import Required Libraries and Set Up Environment Variables

In [510]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import pandas as pd
from pandas import json_normalize

In [511]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [512]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build the complete query URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
print(query_url)

# response = requests.get(query_url)
# print(response.json())

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=XIRmH50X8on3NJp2pA6W03giSAQWFQ92&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531


In [513]:
# Create an empty list to store the reviews
reviews_list = []

# Loop through pages 0-19
for page_num in range(20):
    # Append page number to the constructed URL
    query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&page={page_num}&limit=10"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    json_response = response.json()

    # Add a twelve-second interval between queries to stay within API query limits
    time.sleep(12)

    try:
        # Check if there are any results for the current page
        if json_response['response']['docs']:
            # If there are results, loop through and save the reviews to the reviews_list
            for doc in json_response["response"]["docs"]:
                reviews_list.append(doc)
            
            # Print the page that was just retrieved
            print(f"Checked page {page_num}")
        else:
            # If there are no results, print a message
            print(f"No results found for page {page_num}.")
    except KeyError as e:
        # print the error and break the loop
        print(f"Error: {e}")
        break


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [514]:
import json

# Preview the first 5 results in JSON format
print(json.dumps(reviews_list[:5], indent=4))



[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [515]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
import pandas as pd
from pandas import json_normalize

# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = json_normalize(reviews_list)

# Display the DataFrame
print(reviews_df)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2017/03/09/movies/the-...   
196  https://www.nytimes.com/2017/03/09/movies/revi...   
197  https://www.nytimes.com/2017/03/02/movies/love...   
198  https://www.nytimes.com/2017/02/16/movies/ever...   
199  https://www.nytimes.com/2017/02/09/movies/kedi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York 

In [516]:
print(reviews_df.columns)

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')


In [517]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

import pandas as pd
import re

# Define a function to extract the title
def extract_title(review):
    try:
        text = review['headline']['main'] 
        start_index = text.find('\u2018') + 1
        end_index = text.find('\u2019 Review')
        if start_index > 0 and end_index > start_index:
            return text[start_index:end_index]
        else:
            return None
    except KeyError:
        return None


# Create a new column "title" by applying the extract_title function
reviews_df['title'] = reviews_df.apply(extract_title, axis=1)

# Display the DataFrame with the new "title" column
print(reviews_df)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2017/03/09/movies/the-...   
196  https://www.nytimes.com/2017/03/09/movies/revi...   
197  https://www.nytimes.com/2017/03/02/movies/love...   
198  https://www.nytimes.com/2017/02/16/movies/ever...   
199  https://www.nytimes.com/2017/02/09/movies/kedi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York 

In [518]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(json.dumps)

print(reviews_df)


                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2017/03/09/movies/the-...   
196  https://www.nytimes.com/2017/03/09/movies/revi...   
197  https://www.nytimes.com/2017/03/02/movies/love...   
198  https://www.nytimes.com/2017/02/16/movies/ever...   
199  https://www.nytimes.com/2017/02/09/movies/kedi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York 

In [519]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
def extract_title(review):
    try:
        headline = review['headline.main']  # Access the 'headline.main' column directly
        match = re.search(r'^(.*?):', headline)
        if match:
            return match.group(1)
        else:
            return None
    except KeyError:
        return None

reviews_df['title'] = reviews_df.apply(extract_title, axis=1)

# Print 
print(reviews_df['title'])

0         ‘The Attachment Diaries’ Review
1                                  Review
2           ‘You Can Live Forever’ Review
3      ‘A Tourist’s Guide to Love’ Review
4        ‘Other People’s Children’ Review
                      ...                
195                                Review
196                                Review
197                                Review
198                                Review
199                                Review
Name: title, Length: 200, dtype: object


### Access The Movie Database API

In [520]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [521]:
# Create an empty list to store the results
TMDB_results = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0
# Create a request counter to sleep the requests after a multiple of 50 requests

# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database

def make_request(title):
    global request_counter
    request_counter += 1
    print(f"Request {request_counter} for movie: {title}")
    
    response = requests.get(url + title + tmdb_key_string)
    
    
    if request_counter % 50 == 0:
        print("Reached multiple of 50 requests. Sleeping for 5 seconds...")
        time.sleep(5)
    
    TMDB_results.append(response.json())

titles = ["Inception", "The Matrix", "Interstellar", "The Dark Knight", "The Batman"]


for title in titles:
    make_request(title)

# Print the results
print(TMDB_results)






    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id
         

        # Make a request for a the full movie details


        # Execute "GET" request with url
        
        
        # Extract the genre names into a list

        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
         
        
        # Print out the title that was found
        
for data in TMDB_results:
    try:
            movie_id = data["results"][0]["id"]
            full_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            response = requests.get(full_movie_url)
            if response.status_code == 200:
                movie_details = response.json()
                genres = [genre["name"] for genre in movie_details["genres"]]
                spoken_languages = [lang["english_name"] for lang in movie_details["spoken_languages"]]
                production_countries = [country["name"] for country in movie_details["production_countries"]]
                movie_data = {
                    "title": movie_details["title"],
                    "genres": genres,
                    "spoken_languages": spoken_languages,
                    "production_countries": production_countries
                    # Add more fields as needed
                }
                TMDB_results.append(movie_data)
                print(f"Title found: {movie_details['title']}")
            else:
                print("Movie details not found.")
    except (IndexError, KeyError):
            print("Movie not found.")


Request 1 for movie: Inception
Request 2 for movie: The Matrix


Request 3 for movie: Interstellar
Request 4 for movie: The Dark Knight
Request 5 for movie: The Batman
[{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg', 'genre_ids': [28, 878, 12], 'id': 27205, 'original_language': 'en', 'original_title': 'Inception', 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.', 'popularity': 160.119, 'poster_path': '/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg', 'release_date': '2010-07-15', 'title': 'Inception', 'video': False, 'vote_average': 8.4, 'vote_count': 35475}, {'adult': False, 'backdrop_path': '/JeGkRdNsOuMrgwBdtB0hp763MU.jpg', 'genre_ids': [18, 53], 'id': 613092, 'original_language': 'es', 'original_title': 'El crack cero', 'overview': 'Madrid, Spain, 1975; shortly after the 

In [522]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(TMDB_results[:5], indent=4))

[
    {
        "page": 1,
        "results": [
            {
                "adult": false,
                "backdrop_path": "/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg",
                "genre_ids": [
                    28,
                    878,
                    12
                ],
                "id": 27205,
                "original_language": "en",
                "original_title": "Inception",
                "overview": "Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \"inception\", the implantation of another person's idea into a target's subconscious.",
                "popularity": 160.119,
                "poster_path": "/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg",
                "release_date": "2010-07-15",
                "title": "Inception",
                "video": false,
                "vote_average": 8.4,
                "vote_c

In [523]:
# Convert the results to a DataFrame
df_tmdb_results = pd.DataFrame(TMDB_results)
df_tmdb_results

,page,results,total_pages,total_results,title,genres,spoken_languages,production_countries
0,1.0,"[{'adult': False, 'backdrop_path': '/8ZTVqvKDQ...",1.0,7.0,NaN,NaN,NaN,NaN
1,1.0,"[{'adult': False, 'backdrop_path': '/8K9qHeM6G...",3.0,53.0,NaN,NaN,NaN,NaN
2,1.0,"[{'adult': False, 'backdrop_path': '/xJHokMblj...",1.0,19.0,NaN,NaN,NaN,NaN
3,1.0,"[{'adult': False, 'backdrop_path': '/dqK9Hag10...",2.0,25.0,NaN,NaN,NaN,NaN
4,1.0,"[{'adult': False, 'backdrop_path': '/b0PlSFdDw...",4.0,79.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Inception,"[Action, Science Fiction, Adventure]","[English, French, Japanese, Swahili]","[United Kingdom, United States of America]"
6,NaN,NaN,NaN,NaN,The Matrix,"[Action, Science Fiction]",[English],[United States of America]
7,NaN,NaN,NaN,NaN,Interstellar,"[Adventure, Drama, Science Fiction]",[English],"[United Kingdom, United States of America]"
8,NaN,NaN,NaN,NaN,The Dark Knight,"[Drama, Action, Crime, Thriller]","[English, Mandarin]","[United Kingdom, United States of America]"
9,NaN,NaN,NaN,NaN,The Batman,"[Crime, Mystery, Thriller]",[English],[United States of America]


### Merge and Clean the Data for Export

In [524]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, df_tmdb_results, on="title", how="outer" )
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.person,byline.organization,title,page,results,total_pages,total_results,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,‘The Attachment Diaries’ Review,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2023-05-04T17:16:45+0000,287.0,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,Review,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.nytimes.com/2018/11/01/movies/they...,Morgan Neville’s movie tells the tumultuous be...,The New York Times,"[{""name"": ""subject"", ""value"": ""Documentary Fil...",2018-11-01T13:56:39+0000,794.0,Review: ‘They’ll Love Me When I’m Dead’ Docume...,None,None,The Legend Behind The Scenes,...,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Review,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.nytimes.com/2018/10/25/movies/burn...,The great South Korean director Lee Chang-dong...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2018-10-25T17:39:04+0000,949.0,"Review: In ‘Burning,’ Love Ignites a Divided W...",None,None,Love Ignites a Divided World,...,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Review,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.nytimes.com/2018/10/11/movies/afte...,This cancer drama focuses less on the high sta...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2018-10-11T11:00:01+0000,268.0,"Review: In ‘After Everything,’ a Young Love Bl...",None,None,After Everything,...,"[{'firstname': 'Teo', 'middlename': None, 'las...",None,Review,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Inception,NaN,NaN,NaN,NaN,"[Action, Science Fiction, Adventure]","[English, French, Japanese, Swahili]","[United Kingdom, United States of America]"
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,The Matrix,NaN,NaN,NaN,NaN,"[Action, Science Fiction]",[English],[United States of America]
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Interstellar,NaN,NaN,NaN,NaN,"[Adventure, Drama, Science Fiction]",[English],"[United Kingdom, United States of America]"
208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,The Dark Knight,NaN,NaN,NaN,NaN,"[Drama, Action, Crime, Thriller]","[English, Mandarin]","[United Kingdom, United States of America]"


In [526]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2018/11/01/movies/they...   
3    https://www.nytimes.com/2018/10/25/movies/burn...   
4    https://www.nytimes.com/2018/10/11/movies/afte...   
..                                                 ...   
205                                                NaN   
206                                                NaN   
207                                                NaN   
208                                                NaN   
209                                                NaN   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Morgan Neville’s movie tells the tumultuous be...  The New York 

In [527]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])
merged_df 

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.original,byline.organization,title,page,results,total_pages,total_results,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,None,‘The Attachment Diaries’ Review,NaN,NaN,NaN,NaN,nan,nan,nan
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2023-05-04T17:16:45+0000,287.0,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,By Jeannette Catsoulis,None,Review,NaN,NaN,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/2018/11/01/movies/they...,Morgan Neville’s movie tells the tumultuous be...,The New York Times,"[{""name"": ""subject"", ""value"": ""Documentary Fil...",2018-11-01T13:56:39+0000,794.0,Review: ‘They’ll Love Me When I’m Dead’ Docume...,None,None,The Legend Behind The Scenes,...,By Manohla Dargis,None,Review,NaN,NaN,NaN,NaN,nan,nan,nan
3,https://www.nytimes.com/2018/10/25/movies/burn...,The great South Korean director Lee Chang-dong...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2018-10-25T17:39:04+0000,949.0,"Review: In ‘Burning,’ Love Ignites a Divided W...",None,None,Love Ignites a Divided World,...,By Manohla Dargis,None,Review,NaN,NaN,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2018/10/11/movies/afte...,This cancer drama focuses less on the high sta...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2018-10-11T11:00:01+0000,268.0,"Review: In ‘After Everything,’ a Young Love Bl...",None,None,After Everything,...,By Teo Bugbee,None,Review,NaN,NaN,NaN,NaN,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Inception,NaN,NaN,NaN,NaN,"['Action', 'Science Fiction', 'Adventure']","['English', 'French', 'Japanese', 'Swahili']","United Kingdom, United States of America"
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,The Matrix,NaN,NaN,NaN,NaN,"['Action', 'Science Fiction']",['English'],United States of America
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Interstellar,NaN,NaN,NaN,NaN,"['Adventure', 'Drama', 'Science Fiction']",['English'],"United Kingdom, United States of America"
208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,The Dark Knight,NaN,NaN,NaN,NaN,"['Drama', 'Action', 'Crime', 'Thriller']","['English', 'Mandarin']","United Kingdom, United States of America"


In [528]:
# Delete duplicate rows and reset index
merged_df = merged_df.applymap(lambda x: x if not isinstance(x, list) else str(x))
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
merged_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.original,byline.organization,title,page,results,total_pages,total_results,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2023-05-25T11:00:03+0000,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,NaN,The Attachment Diaries,...,By Jeannette Catsoulis,NaN,‘The Attachment Diaries’ Review,NaN,NaN,NaN,NaN,nan,nan,nan
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2023-05-04T17:16:45+0000,287.0,Review: ‘What’s Love Got to Do With It?’ Proba...,None,NaN,What’s Love Got to Do With It?,...,By Jeannette Catsoulis,NaN,Review,NaN,NaN,NaN,NaN,nan,nan,nan
2,https://www.nytimes.com/2018/11/01/movies/they...,Morgan Neville’s movie tells the tumultuous be...,The New York Times,"[{""name"": ""subject"", ""value"": ""Documentary Fil...",2018-11-01T13:56:39+0000,794.0,Review: ‘They’ll Love Me When I’m Dead’ Docume...,None,NaN,The Legend Behind The Scenes,...,By Manohla Dargis,NaN,Review,NaN,NaN,NaN,NaN,nan,nan,nan
3,https://www.nytimes.com/2018/10/25/movies/burn...,The great South Korean director Lee Chang-dong...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2018-10-25T17:39:04+0000,949.0,"Review: In ‘Burning,’ Love Ignites a Divided W...",None,NaN,Love Ignites a Divided World,...,By Manohla Dargis,NaN,Review,NaN,NaN,NaN,NaN,nan,nan,nan
4,https://www.nytimes.com/2018/10/11/movies/afte...,This cancer drama focuses less on the high sta...,The New York Times,"[{""name"": ""subject"", ""value"": ""Movies"", ""rank""...",2018-10-11T11:00:01+0000,268.0,"Review: In ‘After Everything,’ a Young Love Bl...",None,NaN,After Everything,...,By Teo Bugbee,NaN,Review,NaN,NaN,NaN,NaN,nan,nan,nan


In [529]:
# Export data to CSV without the index
merged_df.to_csv('cleaned_data.csv', index=False)